In [10]:
from typing import List
def get_absolute_file_paths(absolute_dir,file_type)->List[str]:
    '''
    absolute_dir: 文件夹
    file_type: "md","json"...
    '''
    json_files = [os.path.join(absolute_dir,f) for f in os.listdir(absolute_dir) if f.endswith(f".{file_type}")]
    return json_files


In [21]:
import os

from magic_pdf.data.data_reader_writer import FileBasedDataWriter, FileBasedDataReader
from magic_pdf.data.dataset import PymuDocDataset
from magic_pdf.model.doc_analyze_by_custom_model import doc_analyze
from magic_pdf.config.enums import SupportedPdfParseMethod

def MinerU(pdf_file_name):
    # args
    # pdf_file_name = "small_ocr.pdf"  # replace with the real pdf path
    name_without_suff = pdf_file_name.split(".")[0]

    # prepare env
    local_image_dir, local_md_dir = "/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/md/images", "/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/md"
    image_dir = str(os.path.basename(local_image_dir))

    os.makedirs(local_image_dir, exist_ok=True)

    image_writer, md_writer = FileBasedDataWriter(local_image_dir), FileBasedDataWriter(
        local_md_dir
    )

    # read bytes
    reader1 = FileBasedDataReader("")
    pdf_bytes = reader1.read(pdf_file_name)  # read the pdf content

    # proc
    ## Create Dataset Instance
    ds = PymuDocDataset(pdf_bytes)

    ## inference
    if ds.classify() == SupportedPdfParseMethod.OCR:
        infer_result = ds.apply(doc_analyze, ocr=True)

        ## pipeline
        pipe_result = infer_result.pipe_ocr_mode(image_writer)

    else:
        infer_result = ds.apply(doc_analyze, ocr=False)

        ## pipeline
        pipe_result = infer_result.pipe_txt_mode(image_writer)

    # ### draw model result on each page
    # infer_result.draw_model(os.path.join(local_md_dir, f"{name_without_suff}_model.pdf"))

    # ### get model inference result
    # model_inference_result = infer_result.get_infer_res()

    # ### draw layout result on each page
    # pipe_result.draw_layout(os.path.join(local_md_dir, f"{name_without_suff}_layout.pdf"))

    # ### draw spans result on each page
    # pipe_result.draw_span(os.path.join(local_md_dir, f"{name_without_suff}_spans.pdf"))

    # ### get markdown content
    # md_content = pipe_result.get_markdown(image_dir)

    ### dump markdown
    pipe_result.dump_md(md_writer, f"{name_without_suff}.md", image_dir)

    # ### get content list content
    # content_list_content = pipe_result.get_content_list(image_dir)

    # ### dump content list
    # pipe_result.dump_content_list(md_writer, f"{name_without_suff}_content_list.json", image_dir)

    # ### get middle json
    # middle_json_content = pipe_result.get_middle_json()

    # ### dump middle json
    # pipe_result.dump_middle_json(md_writer, f'{name_without_suff}_middle.json')
    print(f"完成{pdf_file_name}")

In [22]:
paths = get_absolute_file_paths("/hpc2hdd/home/fye374/ZWZ_Other/quizmanus/dataset/课本md/高中/ori/高中地理","pdf")

In [23]:
for path in paths:  
    MinerU(path)

2025-04-18 14:07:09.812 | INFO     | magic_pdf.data.dataset:__init__:157 - lang: None
2025-04-18 14:07:14.505 | INFO     | magic_pdf.libs.pdf_check:detect_invalid_chars:67 - cid_count: 264, text_len: 6692, cid_chars_radio: 0.056896551724137934
2025-04-18 14:07:14.521 | WARNING  | magic_pdf.filter.pdf_classify_by_type:classify:335 - OCR needed based on classification result, by_image_area: True, by_text: True, by_avg_words: True, by_img_num: True, by_img_narrow_strips: True, by_invalid_chars: False
2025-04-18 14:07:25.609 | INFO     | magic_pdf.model.doc_analyze_by_custom_model:may_batch_image_analyze:275 - gpu_memory: 79 GB, batch_ratio: 16
Table Predict:   6%|▋         | 1/16 [00:01<00:29,  1.94s/it]